# Collect revert rate data

Method: Use EditAttemptStep and mediawiki_history to identify completed new topic reverts that are reverted within 48 hours. 

In [35]:
import pandas as pd
import numpy as np
import math

import datetime as dt

from wmfdata import hive, mariadb

We will first begin by finding all completed new topic attempts by both test groups in the AB Test.

In [76]:
query = '''

--find all edit attempts
WITH edit_attempts AS (
    SELECT
      wiki AS wiki,
      event.user_id as user_id,
      event.editing_session_id as edit_attempt_id,
      event.bucket AS experiment_group,
      event.is_oversample AS is_oversample,
      event.integration AS editing_method,
      If(event.integration == 'discussiontools', 1, 0) AS new_topic_tool_used,
      CASE
           WHEN event.init_type = 'section' AND event.integration == 'discussiontools' THEN 'new_topic_tool'
           WHEN event.init_type = 'section' AND event.integration == 'page' AND event.init_mechanism IN ('url-new', 'new') THEN 'new_section_link'
          ELSE 'NA' -- check to make sure all edit types accounted for in above list
          END AS section_edit_type,
          event.user_editcount AS experience_level
    FROM event.editattemptstep
WHERE
-- only in participating wikis
    wiki IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
    'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
    'ukwiki', 'viwiki')
-- since deployment
  AND year = 2022
     AND ((month = 01 and day >= 27) OR (month = 02) OR
 (month = 03 and day <= 25))
  -- remove bots
  AND useragent.is_bot = false
-- look at only desktop events
    AND event.platform = 'desktop'
-- review all talk namespaces
    AND event.page_ns % 2 = 1
-- only users in AB test
    AND event.bucket IN ('test', 'control')
-- only registered user
  AND event.user_id != 0
  AND event.action = 'init'
-- discard VE/Wikieditor edits to create new page or reply tool edits
  AND NOT (
 -- not a reply tool edit
  (event.init_type = 'page' AND event.integration = 'discussiontools') OR
 -- not an wikitext edit to create a new page
   (event.init_type = 'page' AND event.init_mechanism IN ('url-new', 'new') AND event.integration = 'page') OR
 -- not a corrective edit to an existing section
   (event.init_type = 'section' AND event.init_mechanism IN ('click', 'url') AND event.integration == 'page') OR
-- not a full page edit
   (event.init_type = 'page' AND event.init_mechanism IN ('click', 'url') AND event.integration = 'page')
 )),

-- find all published comments
published_dt_new_topics AS (
    SELECT
    performer.user_id AS user_id,
    session_id AS edit_save_id,
    revision_id AS revision_id,
    `database` AS wiki
    FROM event.mediawiki_talk_page_edit
    WHERE
    year = 2022
   AND ((month = 01 and day >= 27) OR (month = 02) OR
 (month = 03 and day <= 25))
   -- only in participating wikis
    AND `database` IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
    'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
    'ukwiki', 'viwiki')
),

published_section_link_new_topics AS (
SELECT
     event.user_id as user_id,
     event.editing_session_id AS edit_save_id,
     event.revision_id AS revision_id,
     wiki AS wiki
     FROM event.editattemptstep
     WHERE
 -- only in participating wikis
     wiki IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
     'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
     'ukwiki', 'viwiki')
     AND year = 2022
      AND ((month = 01 and day >= 27) OR (month = 02) OR
 (month = 03 and day <= 25))
   AND event.action = 'saveSuccess'
)


-- main query
SELECT
    eas.wiki,
    eas.user_id,
    edit_attempt_id,
    experiment_group,
    IF(tpe_save.revision_id IS NOT NULL, tpe_save.revision_id, eas_save.revision_id) AS revision_id,
    is_oversample,
    editing_method,
    new_topic_tool_used,
    section_edit_type,
    IF ((section_edit_type = 'new_topic_tool' AND tpe_save.edit_save_id IS NOT NULL)
        OR (section_edit_type = 'new_section_link' AND eas_save.edit_save_id IS NOT NULL), 1, 0) AS edit_success,
    CASE
          WHEN min(experience_level) is NULL THEN 'undefined'
          WHEN min(experience_level) < 100 THEN 'junior'
          ELSE 'non-junior'
          END AS experience_level
FROM edit_attempts eas
LEFT JOIN published_dt_new_topics tpe_save ON
    eas.edit_attempt_id = tpe_save.edit_save_id AND
    eas.wiki = tpe_save.wiki
LEFT JOIN published_section_link_new_topics eas_save ON
    eas.edit_attempt_id = eas_save.edit_save_id AND
    eas.wiki = eas_save.wiki
GROUP BY
    eas.wiki,
    eas.user_id,
    edit_attempt_id,
    IF(tpe_save.revision_id IS NOT NULL, tpe_save.revision_id, eas_save.revision_id),
    experiment_group,
    editing_method, 
    is_oversample,
    new_topic_tool_used,
    section_edit_type,
     IF ((section_edit_type = 'new_topic_tool' AND tpe_save.edit_save_id IS NOT NULL)
        OR (section_edit_type = 'new_section_link' AND eas_save.edit_save_id IS NOT NULL), 1, 0)
'''

In [77]:
new_topic_attempts = hive.run(query)

In [78]:
# only saved edits
completed_edits = new_topic_attempts[new_topic_attempts['edit_success'] == 1] 


We will then be using the mediawiki_history table to identify which of these revisions have been reverted.

In [79]:
query = '''

SELECT
     wiki_db AS wiki,
     event_user_id AS user_id,
      CASE
        WHEN min(event_user_revision_count) is NULL THEN 'undefined'
        WHEN min(event_user_revision_count) < 100 THEN 'junior'
        ELSE 'non-junior'
        END AS experience_level,
    IF(ARRAY_CONTAINS(revision_tags, 'discussiontools-newtopic'), 'new-topic-tool', 'non-new-topic-tool') AS section_edit_type,
     SUM(CAST(
            revision_is_identity_reverted AND 
            revision_seconds_to_identity_revert <= 172800  -- 48 hours
           AS int)) AS num_reverts,
    COUNT(*) as num_comments
FROM wmf.mediawiki_history mwh
WHERE 
    snapshot = '2022-04'
-- look through attempts logged in AB Test recorded in the EditAttemptStep
    AND revision_id IN ({revisions})
     -- find all edits on talk pages
    AND page_namespace_historical % 2 = 1
    AND event_entity = 'revision'
    AND event_type = 'create'
    -- dates of the AB Test
    AND event_timestamp >= '2022-01-27' 
    AND event_timestamp <= '2022-03-25'
    -- on all participating wikis
    AND wiki_db IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
        'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
         'ukwiki', 'viwiki')
    -- user is not a bot and not anonymous
    AND SIZE(event_user_is_bot_by_historical) = 0 
    AND SIZE(event_user_is_bot_by) = 0
    AND event_user_is_anonymous = FALSE
GROUP BY 
 wiki_db,
 event_user_id,
 IF(ARRAY_CONTAINS(revision_tags, 'discussiontools-newtopic'), 'new-topic-tool', 'non-new-topic-tool')
'''

In [80]:
revisions_list = ','.join([str(u) for u in completed_edits["revision_id"]])

In [81]:
new_topic_reverts = hive.run(query.format(revisions=revisions_list))

In [64]:
new_topic_reverts.to_csv('new_topic_reverts.csv', index = False)

# Collect Blocked User Data

In [82]:
query = '''
-- find users that made at least one edit with the new topic tool
WITH topic_users AS (
SELECT
    event_user_id as user_id,
    wiki_db as wiki,
    min(event_timestamp) as first_post_time,
    CASE
        WHEN min(event_user_revision_count) is NULL THEN 'undefined'
        WHEN min(event_user_revision_count) < 100 THEN 'junior'
        ELSE 'non-junior'
        END AS experience_level,
    IF(ARRAY_CONTAINS(revision_tags, 'discussiontools-newtopic'), 'new-topic-tool', 'non-new-topic-tool') AS section_edit_type
FROM wmf.mediawiki_history 
WHERE 
    snapshot = '2022-04'
    AND revision_id IN ({revisions})
     -- find all edit on talk pages
    AND page_namespace_historical % 2 = 1
    AND event_entity = 'revision'
    AND event_type = 'create'
    -- dates of the AB Test 
    AND event_timestamp >= '2022-01-27' 
    AND event_timestamp <= '2022-03-25'
    -- on all participating wikis
    AND wiki_db IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
        'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
         'ukwiki', 'viwiki')
    AND SIZE(event_user_is_bot_by_historical) = 0 
    AND SIZE(event_user_is_bot_by) = 0
    AND event_user_is_anonymous = FALSE
GROUP BY
    event_user_id,
    wiki_db,
    IF(ARRAY_CONTAINS(revision_tags, 'discussiontools-newtopic'), 'new-topic-tool', 'non-new-topic-tool')
),
--find users that are blocked sitewide
blocked_users AS (
SELECT 
    h1.user_id AS blocked_user,
    h1.wiki_db AS blocked_wiki,
    min(h1.start_timestamp) AS block_time 
FROM(
    SELECT *
    FROM wmf.mediawiki_user_history
WHERE 
    snapshot = '2022-03'
    AND start_timestamp >= '2022-01-27' 
    AND start_timestamp <= '2022-03-25'
    AND wiki_db IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
        'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
         'ukwiki', 'viwiki')
    AND caused_by_event_type = 'alterblocks'
    AND inferred_from IS NULL) as h1
LEFT JOIN (
SELECT * FROM wmf.mediawiki_user_history
    WHERE 
    snapshot = '2022-03'
    AND end_timestamp >= '2022-01-27' 
    AND end_timestamp <= '2022-03-25'
    AND wiki_db IN ('amwiki', 'bnwiki', 'zhwiki', 'nlwiki', 'arzwiki', 'frwiki', 'hewiki', 'hiwiki',
        'idwiki', 'itwiki', 'jawiki', 'kowiki', 'omwiki', 'fawiki', 'plwiki', 'ptwiki', 'eswiki', 'thwiki',
         'ukwiki', 'viwiki')
    AND caused_by_event_type = 'alterblocks'
    AND inferred_from IS NULL) AS h2
ON (h1.wiki_db = h2.wiki_db
    AND h1.user_id = h2.user_id
    AND h1.start_timestamp = h2.end_timestamp)
WHERE h2.start_timestamp IS NULL
GROUP BY h1.wiki_db, h1.user_id
)

-- Main Query --
SELECT
    wiki AS wiki,
    experience_level AS experience_level,
    section_edit_type AS section_edit_type,
    SUM(CAST(blocked_user IS NOT NULL and first_post_time < block_time AS int)) AS blocked_user,
    COUNT(*) AS all_users

FROM (
SELECT
    topic_users.first_post_time,
    blocked_users.block_time,
    topic_users.wiki,
    blocked_users.blocked_user,
    topic_users.experience_level,
    topic_users.section_edit_type
FROM topic_users
LEFT JOIN blocked_users ON 
    topic_users.user_id = blocked_users.blocked_user AND
    topic_users.wiki = blocked_users.blocked_wiki 
) sessions
GROUP BY
    wiki,
    experience_level,
    section_edit_type
'''

In [83]:
new_topic_blocks = hive.run(query.format(revisions=revisions_list))

In [84]:
new_topic_blocks.to_csv('new_topic_blocks.csv', index = False)